In [1]:
import os
import sys
import sqlite3
import numpy as np
import pandas as pd
import geopandas as gp
import plotly.express as px
import matplotlib.pyplot as plt
import scipy.stats as scs

import pyet
import pyeto
import spei

In [2]:
sys.path.append('/home/pooya/w/DroughtMonitoringIran/')

GEO_DATA_PATH = "./assets/geo_data"
DATABASE_PATH = "./database/database.db"

In [3]:
conn = sqlite3.connect(DATABASE_PATH)

monthly_data = pd.read_sql(sql='SELECT * FROM ground_data_monthly', con=conn)
monthly_data['Date'] = pd.to_datetime(monthly_data['Date'])

monthly_PET = pd.read_sql(sql='SELECT * FROM pet_monthly', con=conn)
monthly_PET['Date'] = pd.to_datetime(monthly_PET['Date'])

geoinfo = pd.read_sql(sql='SELECT * FROM ground_data_geoinfo', con=conn)

conn.close()

In [ ]:
monthly_data

,Station_ID,Date,Temp_Max,Temp_Min,Temp_Mean,Wind_Speed,Pressure,Humidity_Max,Humidity_Min,Humidity,Sunshine,Precip
0,40709,2006-01-31,6.8,0.9,3.8,0.9,1026.7,96.6,70.3,85.1,2.3,134.46
1,40709,2006-02-28,9.2,3.2,6.2,1.1,1020.6,96.6,78.6,88.8,2.9,37.58
2,40709,2006-03-31,13.2,6.8,10.0,1.7,1019.1,95.6,70.6,84.4,3.2,85.78
3,40709,2006-04-30,16.5,10.9,13.7,0.8,1017.7,96.4,73.8,86.9,3.1,99.07
4,40709,2006-05-31,21.0,14.7,17.9,1.9,1018.8,93.1,67.7,81.3,6.0,39.90
...,...,...,...,...,...,...,...,...,...,...,...,...
5570,99361,2024-03-31,12.1,1.8,6.9,2.0,819.8,80.0,41.8,59.0,6.4,13.60
5571,99361,2024-04-30,15.7,6.2,10.9,2.1,822.6,83.8,48.4,63.0,7.7,48.61
5572,99361,2024-05-31,18.0,9.3,13.6,1.9,821.0,87.9,61.4,75.2,5.3,49.11
5573,99361,2024-06-30,23.2,13.8,18.5,2.0,822.2,86.9,55.9,69.9,7.4,158.21


In [ ]:
monthly_PET

,Station_ID,Date,PET_Hargreaves
0,40709,2006-01-31,23.76
1,40709,2006-02-28,30.78
2,40709,2006-03-31,55.38
3,40709,2006-04-30,70.81
4,40709,2006-05-31,103.12
...,...,...,...
5570,99361,2024-03-31,67.21
5571,99361,2024-04-30,88.16
5572,99361,2024-05-31,107.19
5573,99361,2024-06-30,128.74


In [6]:
data_si = monthly_data\
    .merge(monthly_PET, on=['Station_ID', 'Date'], how='outer')\
    .set_index(['Date'])[['Station_ID', 'Precip', 'PET_Hargreaves']]

data_si['PE_Hargreaves'] = data_si['Precip'] - data_si['PET_Hargreaves']


timescale = [1, 3, 6, 9, 12, 15, 18, 21, 24]
i = 1

for ts in timescale:
    df_spi = data_si\
        .groupby(by='Station_ID')\
        .apply(
            lambda x: spei.spi(
                series=x.Precip,
                dist=scs.gamma,
                prob_zero=True,
                timescale=ts
            ),
            include_groups=False
        )\
        .reset_index()\
        .rename(columns={0: f'SPI_{ts}'})
    if i == 1:
        results = df_spi.copy()
    else:
        results = results.merge(df_spi, on=['Station_ID', 'Date'], how='outer')
    
    df_spei = data_si\
        .groupby(by='Station_ID')\
        .apply(
            lambda x: spei.spei(
                series=x.PE_Hargreaves,
                dist=scs.fisk,
                prob_zero=True,
                timescale=ts
            ),
            include_groups=False
        )\
        .reset_index()\
        .rename(columns={0: f'SPEI_{ts}'})
    results = results.merge(df_spei, on=['Station_ID', 'Date'], how='outer')

    i += 1


results

,Station_ID,Date,SPI_1,SPEI_1,SPI_3,SPEI_3,SPI_6,SPEI_6,SPI_9,SPEI_9,SPI_12,SPEI_12,SPI_15,SPEI_15,SPI_18,SPEI_18,SPI_21,SPEI_21,SPI_24,SPEI_24
0,40709,2006-01-31,0.981495,1.109499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40709,2006-02-28,-1.248436,-1.245763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,40709,2006-03-31,-0.176479,-0.168381,-0.120994,-0.316021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,40709,2006-04-30,0.765971,0.612209,-0.316573,-0.232940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,40709,2006-05-31,0.197567,-0.026885,-0.021366,0.111936,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5518,99361,2024-02-29,0.910169,0.882873,-0.030903,-0.400525,0.026118,-0.220429,0.322641,-0.123471,-0.255763,-0.818338,-0.720390,-1.047768,-1.240657,-1.509941,-1.553530,-1.801988,-1.851724,-1.803042
5519,99361,2024-03-31,-1.362875,-1.502231,-0.207930,-0.490059,-0.375839,-0.885969,-0.220952,-0.932412,-0.037867,-0.978853,-0.970189,-1.355926,-1.499701,-1.617356,-1.597116,-1.836452,-1.815142,-1.950869
5520,99361,2024-04-30,0.619880,0.300687,0.073097,-0.087641,-0.402727,-0.798317,-0.061764,-0.765620,-0.222414,-0.672276,-0.619131,-1.098255,-1.249566,-1.440163,-1.478585,-1.720265,-1.692355,-1.908067
5521,99361,2024-05-31,0.773622,0.796746,-0.456775,-0.567878,-0.160645,-0.476082,-0.082741,-0.402092,0.070161,-0.326494,-0.374709,-0.888041,-0.784768,-1.102732,-1.227741,-1.520228,-0.379947,-1.893520


In [7]:
conn = sqlite3.connect(DATABASE_PATH)

results.to_sql('si_monthly', conn, if_exists='replace', index=False)

conn.commit()
conn.close()